In [1]:
import importlib

if importlib.util.find_spec('transformers') is None:
    print('Installing transformers...')
    ! pip install transformers
    ! pip install pyctcdecode
    ! pip install https://github.com/kpu/kenlm/archive/master.zip
else:
    print('libraries is already installed')

Installing transformers...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.8/414.8 kB 1.8 MB/s eta 0:00:00
     - 553.5 kB 7.5 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.0.0-cp310-cp310-linux_x86_64.whl size=3255175 sha256=9af483ee65a92eb3641536fe46520dc4c2eba19e8b1bcc02b2aee90524ab93c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-gvhrro2b/wheels/a5/73/ee/670fbd0cee8f6f0b21d10987cb042291e662e26e1a07026462
Successfully built kenlm


In [2]:
from pathlib import Path
# from scipy.io import wavfile
# import scipy.signal
import pandas as pd
from tqdm.auto import tqdm
# import seaborn as sns
# import matplotlib.pyplot as plt
# from collections import Counter
import numpy as np
# import os
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Reshape
# from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.regularizers import l1, l2
from transformers import AutoProcessor, TFWav2Vec2Model
from tensorflow.keras import mixed_precision

## Mount Drive

In [3]:
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

ON_COLAB = is_running_on_colab()
ON_COLAB

True

In [4]:
if ON_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  intermediate_folder = Path('/content/gdrive/MyDrive/Colab Notebooks/Speech recognition')
  # intermediate_folder = Path('/content/gdrive/MyDrive/Temp/Speech recognition project')
else:
  intermediate_folder = Path('..') / 'data' / 'intermediate'

Mounted at /content/gdrive


## Read and define model

In [5]:
SAMPLE_RATE = 16000

In [6]:
# model_name = 'jonatasgrosman/wav2vec2-large-xlsr-53-english'
model_name = 'facebook/wav2vec2-base-960h'
processor = AutoProcessor.from_pretrained(model_name)
model = TFWav2Vec2Model.from_pretrained(model_name)


TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing TFWav2Vec2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFWav2Vec2Model were not initialized from the PyTorch model and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

In [7]:
for layer in model.layers:
  layer.trainable = False
  print('Set a layer')

Set a layer


In [8]:
input_ids = tf.keras.Input(shape=(SAMPLE_RATE,), dtype='int32')
attention_masks = tf.keras.Input(shape=(SAMPLE_RATE,), dtype='int32')
output = model([input_ids, attention_masks])
output = output[0]
output = Reshape((49, 768, 1))(output)
output = Conv2D(4, (49, 1), activation='relu', padding='same')(output)
output = MaxPooling2D((49, 1))(output)
output = Flatten()(output)
output = Dropout(.5)(output)
output = Dense(20, activation='softmax')(output)
model_full = tf.keras.models.Model(inputs = [input_ids, attention_masks], outputs=output)
model_full.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 tf_wav2_vec2_model (TFWav2Vec2  TFWav2Vec2BaseModel  94371712   ['input_1[0][0]',                
 Model)                         Output(last_hidden_               'input_2[0][0]']                
                                state=(None, 49, 76                                               
                                8),                                                           

## Read data

In [9]:
X_train = np.load(intermediate_folder / 'train_main_1_sec_audio.npy')
X_train.shape

(33566, 16000)

In [10]:
y_train_labels = pd.read_csv(intermediate_folder / 'train_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_train_labels.shape

(33566,)

In [11]:
X_train_input = processor(X_train, return_tensors="tf", sampling_rate=SAMPLE_RATE).input_values
del X_train
X_train_input.shape

TensorShape([33566, 16000])

In [12]:
X_val = np.load(intermediate_folder / 'val_main_1_sec_audio.npy')
X_val.shape

(4619, 16000)

In [13]:
y_val_labels = pd.read_csv(intermediate_folder / 'val_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_val_labels.shape

(4619,)

In [14]:
X_val_input = processor(X_val, return_tensors="tf", sampling_rate=SAMPLE_RATE).input_values
del X_val
X_val_input.shape

TensorShape([4619, 16000])

In [15]:
# X_test = np.load(intermediate_folder / 'test_main_1_sec_audio.npy')
# X_test.shape

In [16]:
# X_test_input = processor(X_test, return_tensors="tf", sampling_rate=SAMPLE_RATE).input_values
# del X_test
# X_test_input.shape

In [17]:
# y_test_labels = pd.read_csv(intermediate_folder / 'test_main_1_sec_labels.csv', header=None, index_col=False)[0]
# y_test_labels.shape

In [18]:
le = LabelEncoder()
le.fit(y_train_labels)

y_train = le.transform(y_train_labels)
y_val = le.transform(y_val_labels)
# y_test = le.transform(y_test_labels)
y_train.shape, y_val.shape
# y_train.shape, y_val.shape, y_test.shape

((33566,), (4619,))

In [19]:
train_ind = np.random.permutation(len(y_train))
X_train = X_train[train_ind]
y_train = y_train[train_ind]
del train_ind
X_train.shape, y_train.shape, y_train.values

NameError: ignored

## Functions

In [ ]:
# def preprocess(data):
#     return processor(inputs=data, return_tensors='tf', sampling_rate=SAMPLE_RATE).input_values

In [ ]:
attention_mask_train = tf.ones([len(X_train_input), SAMPLE_RATE], dtype=tf.int32)
attention_mask_val = tf.ones([len(X_val_input), SAMPLE_RATE], dtype=tf.int32)

def train(model,
          patience_train,
          patience_val=None,
          learning_rate=.001,
          X_train=X_train_input,
          y_train=y_train,
          X_val=X_val_input,
          y_val=y_val,
          epochs=20,
          batch_size=128,
          checkpoint_file_name='best_model.ckp',
          cont_train=False):
  print(f'{patience_train=}, {patience_val=}, {learning_rate=}, {epochs=}, {batch_size=}, {cont_train=}, {X_train.shape=}, {y_train.shape=}, {X_val.shape=}, {y_val.shape=}')
  model.summary()
  print(f'Input shape {model.input_shape}, output shape {model.output_shape}')
  callbacks = []
  if patience_train:
    callbacks.append(EarlyStopping(monitor='loss', patience=patience_train))
  if patience_val:
    callbacks.append(EarlyStopping(monitor='loss', patience=patience_val))

  callbacks.append(tf.keras.callbacks.ModelCheckpoint(
      filepath=(intermediate_folder / checkpoint_file_name),
      verbose=1,
      save_weights_only=True,
      save_freq='epoch',  # you can also use integer to specify after how many batches to save
      save_best_only=True  # if True, only the best model (lowest validation loss) is saved
      ))

  if not cont_train:
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  model.fit([X_train, attention_mask_train], y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=([X_val, attention_mask_val], y_val),
            callbacks=callbacks);

## Preprocess data

In [ ]:
# X_train_input = map(preprocess, X_train)
# X_val_input = map(preprocess, X_val)
# # X_test_input = map(preprocess, X_test)

In [ ]:
# tf.data.Dataset.from_tensors((X_train, y_train)).map_and_batch_fn(
#     preprocess, batch_size=128)

## Train

In [24]:
train(model=model_full,
      patience_train=15,
      patience_val=15,
      learning_rate=.1,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=15, patience_val=15, learning_rate=0.1, epochs=1000, batch_size=256, cont_train=False, X_train.shape=TensorShape([33566, 16000]), y_train.shape=(33566,), X_val.shape=TensorShape([4619, 16000]), y_val.shape=(4619,)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 tf_wav2_vec2_model (TFWav2Vec2  TFWav2Vec2BaseModel  94371712   ['input_1[0][0]',                
 Model)                         Output(last_hidden_            

KeyboardInterrupt: ignored

In [25]:
train(model=model_full,
      patience_train=15,
      patience_val=15,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=15, patience_val=15, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=TensorShape([33566, 16000]), y_train.shape=(33566,), X_val.shape=TensorShape([4619, 16000]), y_val.shape=(4619,)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 tf_wav2_vec2_model (TFWav2Vec2  TFWav2Vec2BaseModel  94371712   ['input_1[0][0]',                
 Model)                         Output(last_hidden_           

KeyboardInterrupt: ignored

In [26]:
train(model=model_full,
      patience_train=15,
      patience_val=15,
      learning_rate=.001,
      epochs=3,
      batch_size=256,
      # cont_train=True,
      )

patience_train=15, patience_val=15, learning_rate=0.001, epochs=1000, batch_size=256, cont_train=False, X_train.shape=TensorShape([33566, 16000]), y_train.shape=(33566,), X_val.shape=TensorShape([4619, 16000]), y_val.shape=(4619,)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 tf_wav2_vec2_model (TFWav2Vec2  TFWav2Vec2BaseModel  94371712   ['input_1[0][0]',                
 Model)                         Output(last_hidden_          

KeyboardInterrupt: ignored

In [24]:
train(model=model_full,
      patience_train=15,
      patience_val=15,
      learning_rate=.0001,
      epochs=1000,
      batch_size=512,
      # cont_train=True,
      )

patience_train=15, patience_val=15, learning_rate=0.0001, epochs=1000, batch_size=512, cont_train=False, X_train.shape=TensorShape([33566, 16000]), y_train.shape=(33566,), X_val.shape=TensorShape([4619, 16000]), y_val.shape=(4619,)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 tf_wav2_vec2_model (TFWav2Vec2  TFWav2Vec2BaseModel  94371712   ['input_1[0][0]',                
 Model)                         Output(last_hidden_         

KeyboardInterrupt: ignored

In [25]:
# model_full.load_weights(intermediate_folder / 'best_model.ckp')

In [ ]:
train(model=model_full,
      patience_train=15,
      patience_val=15,
      learning_rate=.0001,
      epochs=1000,
      batch_size=512,
      # cont_train=True,
      )

## End

In [ ]:
X_train_input = processor(X_train, return_tensors="tf", sampling_rate=SAMPLE_RATE).input_values
del X_train
X_train_input.shape

TensorShape([33566, 16000])

In [ ]:
@tf.function(jit_compile=True)
def predict_fn(model, input_data):
    return model(input_data)['last_hidden_state']

In [ ]:
mixed_precision.set_global_policy('mixed_float16')

In [ ]:
X_train_output_list = []
for i in tqdm(range(len(X_train_input))):
    X_train_output_list.append(predict_fn(model, tf.reshape(X_train_input[i], (1, -1))))

# Combine all batch predictions
X_train_output = np.concatenate(X_train_output_list)
del X_train_input
del X_train_output_list
X_train_output.shape

  0%|          | 0/33566 [00:00<?, ?it/s]

(33566, 49, 768)

In [ ]:
# BATCH_SIZE = 32  # Define your batch size

# X_train_output_list = []
# for i in tqdm(range(0, len(X_train_input), BATCH_SIZE)):
#     batch_data = X_train_input[i : i + BATCH_SIZE]
#     batch_pred = predict_fn(model, batch_data)
#     X_train_output_list.append(batch_pred)

# # Combine all batch predictions
# X_train_output = np.concatenate(X_train_output_list)
# # del X_train_input
# # del X_train_output_list
# X_train_output.shape

In [ ]:
np.save(intermediate_folder / 'train_main_1_sec_wav2vec_emb.npy', X_train_output)
del X_train_output

In [ ]:
X_val_input = processor(X_val, return_tensors="tf", sampling_rate=SAMPLE_RATE).input_values
del X_val
X_val_input.shape

TensorShape([4619, 16000])

In [ ]:
# X_val_output = model.predict(X_val_input, batch_size=BATCH_SIZE)
# del X_val_input
# X_val_output.shape

In [ ]:
X_val_output_list = []
for i in tqdm(range(len(X_val_input))):
    X_val_output_list.append(predict_fn(model, tf.reshape(X_val_input[i], (1, -1))))

# Combine all batch predictions
X_val_output = np.concatenate(X_val_output_list)
del X_val_input
del X_val_output_list
X_val_output.shape

  0%|          | 0/4619 [00:00<?, ?it/s]

(4619, 49, 768)

In [ ]:
np.save(intermediate_folder / 'val_main_1_sec_wav2vec_emb.npy', X_val_output)
del X_val_output

In [ ]:
X_test_input = processor(X_test, return_tensors="tf", sampling_rate=SAMPLE_RATE).input_values
del X_test
X_test_input.shape

TensorShape([4689, 16000])

In [ ]:
# X_test_output = model.predict(X_test_input, batch_size=BATCH_SIZE)
# del X_test_input
# X_test_output.shape

In [ ]:
X_test_output_list = []
for i in tqdm(range(len(X_test_input))):
    X_test_output_list.append(predict_fn(model, tf.reshape(X_test_input[i], (1, -1))))

# Combine all batch predictions
X_test_output = np.concatenate(X_test_output_list)
del X_test_input
del X_test_output_list
X_test_output.shape

  0%|          | 0/4689 [00:00<?, ?it/s]

(4689, 49, 768)

In [ ]:
np.save(intermediate_folder / 'test_main_1_sec_wav2vec_emb.npy', X_test_output)
del X_test_output